# Matrix Factorization: Toy Example

This is a quick demo for concepts related to Matrix Factorization for Recommender Systems, inspired by the [Colab Notebook](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/recommendation-systems/recommendation-systems.ipynb) of the ["Recommendation Systems" course](https://developers.google.com/machine-learning/recommendation) created by some kind folks at Google. It uses Google's TensorFlow library.

## Scenario
The goal in Matrix Factorization is to find two matrices, a user matrix $U$ and an item matrix $V$, each having the same number of columns. Multiplied together, those two matrices should approximate some "target matrix" $A$, i.e. those two matrices are *factors* that should (roughly) reproduce $A$ when multiplied together. [This](https://youtu.be/ZspR5PZemcs) video explains the general concept very nicely.

The items and users are described by so-called *embeddings*. Note that the "variables" used for embeddings are the same for both the users and the items! In our case, they are very simple: just three numbers. 

What could those numbers mean? This is very context-dependent. In the case of a movie platform they could be "movie/user profiles", for example three common movie categories. A higher value for a particular variable for a user in $U$ could mean that the user likes this particular type of movie a lot. Similarly, a high value for a particular variable for a movie in $V$ could mean that this movie fits the corresponding genre especially well. 

In our example, both $U$ and $V$ have $d = 3$ columns, while $U$ has $n$ rows and $V$ has $m$ rows. Consequently, $A$ (which should be approximated by $U \cdot V^T$) has shape $n \times m$.

### Import required libraries

In [1]:
# required: TensorFlow 2
import tensorflow as tf

### Initialize matrices

We define very simple example matrices for the rating matrix $A$ and its "factor matrices", $U$ and $V$:

In [2]:
# we need to turn on eager execution to be able to see resulting tensors "live", without needing to call eval() (by calling numpy() on the computed tensors)
# more on eager execution and graphs: https://www.tensorflow.org/guide/intro_to_graphs
tf.config.run_functions_eagerly(False)

U = tf.constant([[3.0, 4.0, 5.0], [2.0, 5.0, 3.0]])
V = tf.constant([[3.0, 3.0, 3.0], [5.0, 1.0, 3.0], [1.0, 5.0, 5.0]])
A = tf.sparse.SparseTensor(
  indices = [[0, 2], [1, 0], [1, 2]],
  values = [44.0, 32.0, 38.0],
  dense_shape = [2, 3])

2022-05-31 17:41:57.340545: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-31 17:41:57.341922: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


While executing TensorFlow code eagerly, Tensor's have a `numpy()` function that we can use to inspect their contents:

In [3]:
U.numpy()

array([[3., 4., 5.],
       [2., 5., 3.]], dtype=float32)

In [4]:
V.numpy()

array([[3., 3., 3.],
       [5., 1., 3.],
       [1., 5., 5.]], dtype=float32)

For printing sparse tensors, we can make use of the following utility function:

In [5]:
def pprint_sparse_tensor(st):
  s = "<SparseTensor\n shape = %s \n values = {" % (st.dense_shape.numpy().tolist(),)
  for (index, value) in zip(st.indices.numpy(), st.values.numpy()):
    s += f"\n  %s: %s," % (index.tolist(), value.tolist())
  print(s + "\n }>")

This is the sparse matrix we would like to approximate with $U \cdot V^T$:

In [6]:
pprint_sparse_tensor(A)

<SparseTensor
 shape = [2, 3] 
 values = {
  [0, 2]: 44.0,
  [1, 0]: 32.0,
  [1, 2]: 38.0,
 }>


## Evaluating quality of Matrix Factorization

In practice, an "ideal" Matrix Factorization is learned by Machine Learning algorithms. As every Machine Learning algorithm needs some loss function to optimize, we will also use one in this example. We will go with the Mean Squared Error (MSE).

### 1. Computing "prediction" (using $U$ and $V$) for existing elements in $A$

This goal is achieved by multiplying $U$ with $V^T$ and keeping only entries that were non-zero in $A$.

#### Simpler (less efficient) approach

Use matrix multiplication and then subset with `tf.gather()`:

In [7]:
matrix_product = U @ tf.transpose(V)
matrix_product.numpy()

array([[36., 34., 48.],
       [30., 24., 42.]], dtype=float32)

In [8]:
# gather_nd() allows us to pick selected elements from a multi-dimensional tensor via a multi-dimensional tensor of indices
result_a = tf.gather_nd(matrix_product, A.indices)

In [9]:
result_a.numpy()

array([48., 30., 42.], dtype=float32)

#### More efficient approach

We only take those pairs of elements from $U$ and $V$ into account that actually appear in the original matrix $A$, so we skip all cases where we would multiply with 0 (or NaN). In practice this makes a lot of sense, as most users have rated only a tiny fraction of all the available items, i.e. the matrix $A$ is very sparse. 

In [10]:
match_indices_U = A.indices[:, 0]
match_indices_V = A.indices[:, 1]

In [11]:
match_indices_U.numpy()

array([0, 1, 1])

In [12]:
match_indices_V.numpy()

array([2, 0, 2])

How does this make sense? Compare with the non-zero indices of $A$:

In [13]:
A.indices.numpy()

array([[0, 2],
       [1, 0],
       [1, 2]])

In [14]:
#  TODO: finish
#  def sparse_2d_tensor_to_dense_2d_array(sparse):
#   matrix = np.ndarray(shape = sparse.dense_shape.numpy())
#   for i, pos in enumerate(sparse.indices.numpy().tolist()):
#     matrix[pos]

So, the elements we would consider are:

In [15]:
# gather() allows us to pick selected elements from a tensor via a tensor of indices
match_partners_U = tf.gather(U, match_indices_U)
match_partners_V = tf.gather(V, match_indices_V)

Now, we do element-wise multiplication:

In [16]:
elementwise = match_partners_U * match_partners_V

In [17]:
elementwise.numpy()

array([[ 3., 20., 25.],
       [ 6., 15.,  9.],
       [ 2., 25., 15.]], dtype=float32)

And, finally, sum by row:

In [18]:
result_b = tf.reduce_sum(elementwise, axis = 1)

In [19]:
result_b.numpy()

array([48., 30., 42.], dtype=float32)

The result is identical to the other approach, nice!

### 2. Computing MSE from prediction

Our prediction was:

In [20]:
prediction = result_a # result_b is identical
prediction.numpy()

array([48., 30., 42.], dtype=float32)

The actual (non-zero) entries in the matrix were:

In [21]:
A.values.numpy()

array([44., 32., 38.], dtype=float32)

To compute the MSE, we now just sum up the squared differences for each vector element and divide by the length of the two vectors (or tensors, to be exact). TensorFlow has a utility function for this:

In [22]:
mse = tf.losses.mean_squared_error(A.values, prediction)

In [23]:
mse.numpy()

12.0

That makes sense, we were off by 2 each time, 3 * 2^2 = 12.

### Putting everything into a function

In [24]:
def sparse_mean_square_error(sparse_ratings, user_embeddings, movie_embeddings):
  """
  Args:
    sparse_ratings: A SparseTensor rating matrix, of dense_shape [N, M]
    user_embeddings: A dense Tensor U of shape [N, k] where k is the embedding
      dimension, such that U_i is the embedding of user i.
    movie_embeddings: A dense Tensor V of shape [M, k] where k is the embedding
      dimension, such that V_j is the embedding of movie j.
  Returns:
    A scalar Tensor representing the MSE between the true ratings and the
      model's predictions.
  """
  predictions = tf.reduce_sum(
      tf.gather(user_embeddings, sparse_ratings.indices[:, 0]) *
      tf.gather(movie_embeddings, sparse_ratings.indices[:, 1]),
      axis=1)
  loss = tf.losses.mean_squared_error(sparse_ratings.values, predictions)
  return loss

sparse_mean_square_error(A, U, V).numpy()

12.0

## Usage of Matrix Factorization in practice

### How to find "best" Matrix Factorization?

In practice, a ML model (most probably a neural network) would "learn" a good matrix factorization over several iterations, each time trying to minimize the loss function. In neural networks, "Stochastic Gradient Descent" is commonly used for that purpose.

### How to use for recommendations?

The whole purpose of Matrix factorization is to use it for predictions on new, unseen data. The matrix factorization $U \cdot V^T$ can be thought of as a "score matrix" for item/user combinations. By learning a good approximation for $A$, we implicitly also learn "expected scores" for combinations of users and items that didn't occur up until now. This means, that for recommending a new item to a user, we can simply have a look at that user's row in $U \cdot V^T$ and pick the element with the highest score! The index of that element is the item we should try to recommend first!

## Using regularization to improve recommendations from Matrix Factorization model

As noted in the the [Colab Notebook](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/recommendation-systems/recommendation-systems.ipynb) of Google's ["Recommendation Systems" course](https://developers.google.com/machine-learning/recommendation), using the MSE on the observed part of the rating matrix $A$ may result in poor model performance, as the model does not learn how to place the embeddings of irrelevant items. This phenomenon is known as *folding*.

We will add regularization terms that will address this issue. We will use two types of regularization:
- Regularization of the model parameters. This is a common $\ell_2$ regularization term on the embedding matrices, given by $r(U, V) =  \frac{1}{N} \sum_i \|U_i\|^2 + \frac{1}{M}\sum_j \|V_j\|^2$.  
*Note: We use $\|a\|$ to denote the *norm* of a vector $a$. So, $r(U,V)$ is actually a penalty on the magnitude of the rows of $U$ and $V$, i. e. the user/item embeddings in the embedding matrices.*
- A global prior that pushes the prediction of any pair towards zero, called the *gravity* term. This is given by $g(U, V) = \frac{1}{MN} \sum_{i = 1}^N \sum_{j = 1}^M \langle U_i, V_j \rangle^2$.

The total loss is then given by
$$
\frac{1}{|\Omega|}\sum_{(i, j) \in \Omega} (A_{ij} - \langle U_i, V_j\rangle)^2 + \lambda _r r(U, V) + \lambda_g g(U, V)
$$
where $\lambda_r$ and $\lambda_g$ are two regularization coefficients (hyper-parameters).

We can express $r(U,V)$ and $g(U, V)$ in code like this:

In [25]:
def regularization(U, V):
  return (tf.reduce_sum(U * U) / (U.shape[0])  + 
          tf.reduce_sum(V * V) / (V.shape[0]))

In [26]:
def gravity(U, V):
  """Creates a gravity loss given two embedding matrices."""
  return 1. / (U.shape[0] * V.shape[0]) * tf.reduce_sum(
      tf.matmul(U, U, transpose_a=True) * tf.matmul(V, V, transpose_a=True))

We would then define the regularization and gravity coefficients ${\lambda}_r$ and ${\lambda}_g$ as hyperparameters before training and compute the loss in each epoch/iteration of the training process with a function that sums the three losses/error terms together to get the "total loss". The computation could look something like this:

In [28]:
regularization_coeff = 0.1
gravity_coeff = 1.0

def get_regularized_loss(A, U, V):
  return sparse_mean_square_error(A, U, V) + regularization_coeff * regularization(U, V) + gravity_coeff * gravity(U, V)

In [29]:
get_regularized_loss(A, U, V).numpy()

1352.8334

We see that the loss is *much* larger, as the values of $U$ and $V$ are quite large. I am also not so sure if the example values I picked for the entries in $A$ even make much sense. It seems to me as if neural network weights are often initialized with values following a standard normal distribution (which is centered around 0)? At least then the *gravity term* that pushes the prediction of any pair towards zero would also make more sense. But I must admit that I don't understand the topic well enough at this point. For a more meaningful discussion of results of non-regularized vs. regularized Matrix Factorization models see also the notebook I reference above. 